# Sumário
**1** <a href='#1-rag-in-pdf-file'> RAG PDF files</a>  
1.2 <a href='#passando-o-resultado-da-consulta-ao-banco-vetorial-manualmente'> Passando-o-resultado-da-consulta-ao-banco-vetorial-manualmente</a>  
1.3 <a href='#adicionar-memória-lce'>adicionar-memória-lce</a>  
**2** <a href='#contruindo-um-assistente-de-documentação'> Assistente de documentação</a>



In [1]:

from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [2]:
# Instância do LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# 1 RAG (Retrieval Augmented Generation) in PDF file

**RAG index**  

<img src='assets/imgs/rag_indexing.png' width=800></img>

In [6]:
# Carregar PDF 
from langchain_community.document_loaders import PyPDFLoader

pdf_path = 'data/pdfs/Tese_Ficha_catalografica[v3].pdf' 
loader = PyPDFLoader(pdf_path)

docs = loader.load()

Ao carregar o PDF é necessário subdividir o arquivo em pequenas seções (chunk), pois, caso o documento seja muito grande pode ultrapassar a janela de contexto do LLM. 

In [7]:
# Criar os chunks com tamanho padrão
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [5]:
# Carrega os documentos no banco vetorial
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(), persist_directory='./db', collection_name='Tese')

# Retrieve 
retriever = vectorstore.as_retriever()

**Retrieval** (recuperação)

<img src='assets/imgs/rag_retrieval_generation.png' width=600></img> 

In [7]:
# Load database
vectorstore = Chroma(persist_directory='./db', collection_name='Tese', embedding_function=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()
# 
retriever.invoke('Qual o município mais vulnerável') 

[Document(metadata={'page': 72, 'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf'}, page_content='regiões mais vulnerávei s, todavia os cartogramas apresentados nesta seção fornece m \numa visão holística do problema , de modo a permitir um direcionamento pontual das \npolíticas públicas  de acordo com as necessidades de cada município.'),
 Document(metadata={'page': 76, 'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf'}, page_content='75 \n \nnegativo para a maioria dos municípios fluminenses. Essa proporção foi semelhante \naos achados de Raiher e Lima (2014) na região Sul do Brasil , com 83 % e 73% dos \nmunicípios no círculo vicio so, respectivamente a média brasileira e da Região Sul; aos \nresultados de Oliveira , Lima e Raiher,  (2017) , na Região Nordeste , 84% dos \nmunicípios  no círculo vicioso quando comparado com a média do Brasil, todavia, 24% \nquando comparado  com a média da própri a região ; e Oliveira , Lima  e Barrinha,  (2019) , \nno Estado da Bahia , 83%

## 1.2 RAG

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | llm

In [6]:
rag_chain.invoke('Qual o município que precisa de atenção urgente?')

AIMessage(content='O município que precisa de atenção urgente é Porto Real, pois apresenta baixos indicadores de desenvolvimento humano e está classificado na categoria de círculo vicioso. Essa classificação indica uma relação negativa com o quociente locacional, o que sugere dificuldades no crescimento econômico.', response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 973, 'total_tokens': 1025, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-687013fb-1990-4b30-98a8-a4d76e95bb73-0', usage_metadata={'input_tokens': 973, 'output_tokens': 52, 'total_tokens': 1025})

### Passando o resultado da consulta ao banco vetorial manualmente

In [43]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | llm
    | StrOutputParser()
)


question = "Qual o municipio que precisa de atenção urgente?"

docs = vectorstore.similarity_search(question)
# Run
chain.invoke({"context": docs, "question": question})


[Document(metadata={'page': 72, 'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf'}, page_content='regiões mais vulnerávei s, todavia os cartogramas apresentados nesta seção fornece m \numa visão holística do problema , de modo a permitir um direcionamento pontual das \npolíticas públicas  de acordo com as necessidades de cada município.'), Document(metadata={'page': 63, 'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf'}, page_content='62 \n \nO município de Macaé, parte superior,  foi o único classificado como tendendo \nao crescimento e tem uma alta relação com o QL, que mede a concentração da \nindústria . Nota-se que Porto Real também está associa do ao quociente locacional, \napesar de se encontrar na categoria  círculo vicioso . A diferença entre esses dois \nmunicípios  refere -se aos indicadores essenciais do desenvolvim ento humano : quanto \nmais abaixo da média, caso de Porto Real, mais próximo do círculo vicioso . Esses \ndois casos ilustram bem a ideia de que não 

'O município que precisa de atenção urgente é Porto Real, que está classificado na categoria de "círculo vicioso" e apresenta indicadores essenciais de desenvolvimento humano abaixo da média. Isso indica uma situação crítica em comparação com Macaé, que tende ao crescimento.'

## Adicionar memória LCE
https://github.com/TirendazAcademy/LangChain-Tutorials

https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/  

<img src='assets/imgs/conversational_retrieval_chain.png' width=900></img>

**Contextualizando a questão**

Esta cadeia adiciona uma reformulação da consulta de entrada ao nosso mecanismo de busca, de modo que a recuperação incorpore o contexto da conversa.

In [4]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage

### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{question}"),
    ]
)

contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [5]:
# Exemplo - retorna pergunta reformulada que será passada para o retriever
contextualize_q_chain.invoke(
    {
        "chat_history":[
            HumanMessage(content="Me liste os os vazamentos"),
            AIMessage(content="É uma distribuição estatística que segue uma distribuição normal"),
        ],
        "question": "Ela também é conhecida como outro nome?",
    }
)

'Ela é conhecida como distribuição gaussiana.'

**QA chain**

In [8]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]
    

rag_chain = (
    RunnablePassthrough.assign(
        context = contextualized_question | retriever | format_docs
    )
    | qa_prompt 
    | llm
)

In [14]:
# Inference

rag_chain.invoke(
    {
        "chat_history":[
            HumanMessage(content="O que é uma curva normal?"),
            AIMessage(content="É uma distribuição estatística que segue uma distribuição normal"),
        ],
        "question": "Ela também é conhecida como outro nome?",
    }
)

AIMessage(content='Sim, a curva normal também é conhecida como distribuição gaussiana.', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 688, 'total_tokens': 702}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-0324f2f3-33be-486d-a183-fb35b312fb18-0', usage_metadata={'input_tokens': 688, 'output_tokens': 14, 'total_tokens': 702})

### Gerenciar memória 

Aqui, vimos como adicionar lógica de aplicativo para incorporar saídas históricas, mas ainda estamos atualizando manualmente o histórico de bate-papo e inserindo-o em cada entrada. Em um aplicativo real de perguntas e respostas, queremos alguma maneira de persistir o histórico de bate-papo e alguma maneira de inseri-lo e atualizá-lo automaticamente.

Para isso, podemos usar:

* [BaseChatMessageHistory](https://python.langchain.com/v0.2/api_reference/langchain/index.html#module-langchain.memory): Armazene o histórico de bate-papo.
* [RunnableWithMessageHistory](https://python.langchain.com/v0.2/docs/how_to/message_history/): wrapper para uma cadeia LCEL e um que lida com a injeção de histórico de chat em entradas e atualizando-o após cada invocação.BaseChatMessageHistory

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """
    Existe varias interfaces para conectar com banco de dados no langchain:
        * langchain-postgres
        * langchain-redis
        * SQLChatMessageHistory (SQLite)

    Args:
        session_id (str): id para identifcar a sessão

    Returns:
        BaseChatMessageHistory: Classe responsável por armazenar o historico
    """
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history",
)

In [10]:
conversational_rag_chain.invoke(
    {"question": "Qual a distribuição que eu citei"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

AIMessage(content='A distribuição mencionada refere-se ao histograma apresentado na Figura 16, onde os municípios classificados como "Outros" tendem aos extremos da distribuição. As variáveis QL, PDA, PDBA e PDCL apresentam uma sobreposição completa das faixas de variação, indicando que não há uma discriminação clara entre as categorias CVic e "Outros".', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 652, 'total_tokens': 728, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-016adf70-9ee5-47ce-a689-ec6805e4a4eb-0', usage_metadata={'input_tokens': 652, 'output_tokens': 76, 'total_tokens': 728})

### Aboradagem usando funções auxiliares 
* create_history_aware_retriever e 
* create_retrieval_chain

In [27]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

rag_chain.invoke(
    {
        "chat_history":[
            HumanMessage(content="O que é uma curva normal?"),
            AIMessage(content="É uma distribuição estatística que segue uma distribuição normal"),
        ],
        "input": "Ela também é conhecida como outro nome?",
    }
)

{'chat_history': [HumanMessage(content='O que é uma curva normal?'),
  AIMessage(content='É uma distribuição estatística que segue uma distribuição normal')],
 'input': 'Ela também é conhecida como outro nome?',
 'context': [Document(metadata={'page': 52, 'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf'}, page_content='51 \n \nFigura 6. Matriz do d iagrama de dispersão das variáveis de estudo  \n \nFonte: Autor.'),
  Document(metadata={'page': 68, 'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf'}, page_content='Fonte: Autor  \n \nNo histograma apresentado na Figura 1 6 nota-se que os munícipios \nclassificados como “Outros ” tendem aos estremos da dis tribuição  e que as variáveis \nQL, PDA, PDBA e PDCL apresentam praticamente uma sobreposição completa das \nfaixas de variação. Des sa maneira, não há uma discriminação das categorias CVic e \n“Outros” para essas variáveis.'),
  Document(metadata={'page': 37, 'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf'}, page_conte

## Contruindo um assistente de documentação 

Obter documentação (bash):
* wget -r -A.html -P langchain-docs https://api.python.langchain.com/en/latest/langchain_api_reference.html 

In [3]:
from langchain.document_loaders import ReadTheDocsLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [6]:
def exist_colecttion(collection_name, persist_directory='db'):
    try: 
        Chroma(
            persist_directory='db', 
            collection_name=collection_name, 
            create_collection_if_not_exists=False
        )
        return True
    except:
        return False

def ingest_docs():
    """Carregar documentação
    Carrega a documentação do langchain no banco vetorial
    """
    loader = ReadTheDocsLoader('data/langchain-docs/api.python.langchain.com/en/latest')
    
    raw_docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
    
    print(f' raw documents: {len(raw_docs)}')
    
    documents = text_splitter.split_documents(raw_docs[0:500])
    
    print(f'docs {len(documents)}')
    
    for doc in documents:
        new_url = doc.metadata['source']
        new_url = new_url.replace("langchain-docs", "https:/")
        doc.metadata.update({'source': new_url})  
        
    collection_name='docs_langchain'
    
    if exist_colecttion(collection_name): 
        return
         
    Chroma.from_documents(
        documents=documents, 
        embedding=embeddings, 
        persist_directory='db', 
        collection_name=collection_name
    )
    
    print(f'Dados persistido no banco')
        
    
    

In [7]:
ingest_docs()

 raw documents: 4142
docs 9378
cole docs_langchain
Dados persistido no banco


**Retrieval Augmentation Generation (RAG)**

In [29]:
from langchain_core.prompts import ChatPromptTemplate

collection_name='docs_langchain'

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def run_llm(query:str):
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
    docsearch = Chroma(
            persist_directory='db', 
            collection_name=collection_name, 
            embedding_function=embeddings
        )
    
    docs = docsearch.similarity_search(query)
    
    chat = ChatOpenAI(model="gpt-4o-mini")
    
    RAG_TEMPLATE = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

    <context>
    {context}
    </context>

    Answer the following question:

    {question}"""

    prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)
    
    
    rag_chain = (
        RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
        | prompt
        | chat
    )
    
    res = rag_chain.invoke({'question':query, 'context': docs}) 
    
    return  res, docs


    
query = 'Whats is a langchain Chain?'
    
run_llm(query)
    

(AIMessage(content='A LangChain Chain is a sequence of actions that is hardcoded to perform specific tasks. Unlike Agents, which use a language model to determine actions dynamically, Chains follow a predefined order of execution. Examples include the PALChain, which implements Program-Aided Language Models, and other specialized chains within the LangChain framework.', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 502, 'total_tokens': 565}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-2e2dc611-f212-4176-b5d0-7a450f8b93d3-0', usage_metadata={'input_tokens': 502, 'output_tokens': 63, 'total_tokens': 565}),
 [Document(metadata={'source': 'data/https://api.python.langchain.com/en/latest/exceptions/langchain_core.exceptions.LangChainException.html'}, page_content='langchain_core.exceptions.LangChainException¶\nclass langchain_core.exceptions.LangChainException[source]¶\nGene

**Usando funções auxiliares: create_retrieval_chain, create_stuff_documents_chain**

In [16]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

def run_llm_(query: str):
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
    docsearch = Chroma(
            persist_directory='db/', 
            collection_name=collection_name, 
            embedding_function=embeddings
        )
    
    chat = ChatOpenAI(model="gpt-4o-mini", temperature=0, verbose=True)
    
    RAG_TEMPLATE = """
        You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

        <context>
        {context}
        </context>

        Answer the following question:

         {question}"""

    retrieval_qa_chat_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

    stuff_documents_chain = create_stuff_documents_chain(chat, retrieval_qa_chat_prompt)

    qa = create_retrieval_chain(
        retriever=docsearch.as_retriever(), combine_docs_chain=stuff_documents_chain
    )
    result = qa.invoke(input={"input": query})
    return result

# OPENSEARCH

In [4]:
from langchain_community.vectorstores import OpenSearchVectorSearch

os = OpenSearchVectorSearch(
        embedding_function=OpenAIEmbeddings(),
        opensearch_url='localhost:9200',
        http_auth=('admin','admin'),
        index_name="tese",
        is_appx_search=False
)

In [5]:
filtros = {
    "bool": {
        "filter": 
            {"match": {"metadata.page": 72}},
        
    }
}

retriever = os.as_retriever(search_kwargs={'filter':filtros})
# 
retriever.invoke('Qual o município mais vulnerável') 

[Document(metadata={'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf', 'page': 72}, page_content='regiões mais vulnerávei s, todavia os cartogramas apresentados nesta seção fornece m \numa visão holística do problema , de modo a permitir um direcionamento pontual das \npolíticas públicas  de acordo com as necessidades de cada município.'),
 Document(metadata={'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf', 'page': 72}, page_content='regiões mais vulnerávei s, todavia os cartogramas apresentados nesta seção fornece m \numa visão holística do problema , de modo a permitir um direcionamento pontual das \npolíticas públicas  de acordo com as necessidades de cada município.'),
 Document(metadata={'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf', 'page': 72}, page_content='normalizadas (escala de 0 a 1), antes e depois da combinação, para a criação do \nindicador. As faixas foram definidas de acordo com a escala do IDH estabelecidas \npelo PNUD.  Dessa maneira foi possível

In [9]:
filter = {"bool": {"filter": {"term": {"metadata.page": 72}}}}
docs = os.similarity_search(
    "vulnerabilidade",
    # search_type="painless_scripting",
    # space_type="cosineSimilarity",
    # pre_filter=filter,
    # efficient_filter=filter
    boolean_filter=filter,
    # metadata_field='metadata'
)
docs

[Document(metadata={'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf', 'page': 72}, page_content='regiões mais vulnerávei s, todavia os cartogramas apresentados nesta seção fornece m \numa visão holística do problema , de modo a permitir um direcionamento pontual das \npolíticas públicas  de acordo com as necessidades de cada município.'),
 Document(metadata={'source': 'data/pdfs/Tese_Ficha_catalografica[v3].pdf', 'page': 72}, page_content='regiões mais vulnerávei s, todavia os cartogramas apresentados nesta seção fornece m \numa visão holística do problema , de modo a permitir um direcionamento pontual das \npolíticas públicas  de acordo com as necessidades de cada município.')]